In [3]:
using CSV
using DataFrames
using Plots

In [4]:
include("./src/utils.jl");

In [5]:
scenario = 11
year = 1998
gen_prop_name = "vivienne_2050"
bus_prop_name = "vivienne_2050"
branch_prop_name = "vivienne_2050"
out_path = "./out/";

In [98]:
# function print_full_model(model, filename)
#     open(filename, "w") do f
#         # Print objective
#         println(f, "Objective: ", objective_function(model))
        
#         # Print all constraints
#         for (F, S) in list_of_constraint_types(model)
#             for con in all_constraints(model, F, S)
#                 println(f, con)
#             end
#         end
#     end
# end

# # Usage
# print_full_model(model, "model.txt")

In [97]:
# df = CSV.read("data/grid/npcc_new.csv", DataFrame);

# plot(df.xcoord, df.ycoord, 
#     zcolor = df.busIdx,
#     seriestype = :scatter,
#     markersize=5,
#     color = palette(:heat, size(df)[1]))